In [1]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
for i in range(100):

    face_path = './dataset/train/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb,1)
    
    img_show_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [3]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['img'+str(i)] = globals()['img'+str(i)][0][48:]
    globals()['img'+str(i)] = globals()['img'+str(i)].flatten()

In [4]:
train_x = np.zeros((100, 34))
for i in range(100):
    for j in range(34):
        train_x[i][j] = globals()['img'+str(i)][j]

In [5]:
for i in range(100):

    face_path = './dataset/test/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb,1)
    
    img_show_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)

    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['img_'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img_'+str(i)].append(list_points)
        globals()['img_'+str(i)] = np.array(globals()['img_'+str(i)])

In [6]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['img_'+str(i)] = globals()['img_'+str(i)][0][48:]
    globals()['img_'+str(i)] = globals()['img_'+str(i)].flatten()
    
test_x = np.zeros((100, 34))
for i in range(100):
    for j in range(34):
        test_x[i][j] = globals()['img_'+str(i)][j]

In [7]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 30:]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 30:]

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)
y_pred = model.predict(train_x)
mae = mean_absolute_error(train_y, y_pred)

mae

0.051221096024850865

In [9]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)

mae

0.07303470861276531

In [10]:
model.intercept_, model.coef_

(array([-18.14846963,  26.50348511,   4.99422284,   7.46871556]),
 array([[-0.02216445, -0.03122309,  0.01750076, -0.01709837, -0.00240221,
          0.01896417,  0.0214418 , -0.02006556, -0.04105427,  0.02156432,
          0.03768198,  0.0389769 , -0.035959  , -0.00658911,  0.01818112,
         -0.00336055,  0.0422829 ,  0.03577747,  0.00511455, -0.05597689,
          0.00584424,  0.00928021, -0.03150037,  0.00156691,  0.01004634,
         -0.0403903 , -0.01219887,  0.03162032, -0.0071621 ,  0.03377449,
          0.01850482, -0.03469126,  0.02250261,  0.01503696],
        [-0.0026315 ,  0.00752317, -0.00116664,  0.00493764,  0.00456382,
         -0.01174099,  0.00640061,  0.01100236,  0.01565655, -0.00714645,
         -0.02922764,  0.00033098,  0.01626805, -0.00744771,  0.00134038,
         -0.00979102, -0.01132086, -0.01585563, -0.00052106,  0.01603896,
          0.00350107,  0.00146183, -0.00360613,  0.00635401, -0.00021214,
         -0.01140869,  0.00097603, -0.02015436, -0.0056934

In [12]:
prediction = pd.DataFrame(y_pred)
prediction.columns = ['mouthSize','mouthPosition', 'mouthPronounced','lipsSize']
prediction

,mouthSize,mouthPosition,mouthPronounced,lipsSize
0,0.273658,0.557839,0.447196,0.630884
1,0.281768,0.695206,0.492050,0.681149
2,0.358863,0.615057,0.335767,0.511513
3,0.415881,0.186695,0.371273,0.265506
4,0.502459,0.852709,0.516442,0.437354
...,...,...,...,...
95,0.579443,0.151522,0.398579,0.547701
96,0.601397,0.457891,0.414464,0.762571
97,0.469399,0.316710,0.667493,0.495769
98,0.673718,0.402725,0.382619,0.665076


In [13]:
test_y

,mouthSize,mouthPosition,mouthPronounced,lipsSize
0,0.241427,0.614988,0.780424,0.520251
1,0.298667,0.704906,0.733134,0.742770
2,0.228592,0.651212,0.317283,0.440955
3,0.481674,0.214611,0.633301,0.222423
4,0.676315,0.793210,0.581524,0.521908
...,...,...,...,...
95,0.549648,0.204712,0.569278,0.322924
96,0.617910,0.485857,0.463260,0.680530
97,0.445559,0.314909,0.787422,0.434210
98,0.660861,0.420872,0.230623,0.670635


In [46]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(test_y, prediction)**0.5
print(rmse)

0.12469882397879001
